In [1]:
!pip install pandas scikit-learn nltk

  Using cached pandas-2.2.2-cp311-cp311-win_amd64.whl (11.6 MB)
  Using cached scikit_learn-1.4.2-cp311-cp311-win_amd64.whl (10.6 MB)
     ---------------------------------------- 1.5/1.5 MB 2.9 MB/s eta 0:00:00
  Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl (15.8 MB)
  Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)
  Using cached scipy-1.13.0-cp311-cp311-win_amd64.whl (46.2 MB)
  Using cached joblib-1.4.0-py3-none-any.whl (301 kB)
  Using cached click-8.1.7-py3-none-any.whl (97 kB)
     ------------------------------------- 269.0/269.0 kB 16.2 MB/s eta 0:00:00
     ---------------------------------------- 78.3/78.3 kB ? eta 0:00:00



[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [2]:
data_pornographic = None
with open('pornogrpahic.json', 'r', encoding='utf-8') as f:
    data_pornographic = json.load(f)

data_harmful = None
with open('harmful.json', 'r', encoding='utf-8') as f:
    data_harmful = json.load(f)

data_porn_en = None
with open('porn.json', 'r', encoding='utf-8') as f:
    data_porn_en = json.load(f)

data_sexual_assault = None
with open('sexual_assault.json', 'r', encoding='utf-8') as f:
    data_sexual_assault = json.load(f)

data = data_pornographic + data_harmful + data_porn_en

In [4]:
data

[{'kind': 'Listing',
  'data': {'after': None,
   'dist': 1,
   'modhash': '',
   'geo_filter': '',
   'children': [{'kind': 't3',
     'data': {'approved_at_utc': None,
      'subreddit': 'rusAskReddit',
      'selftext': '',
      'user_reports': [],
      'saved': False,
      'mod_reason_title': None,
      'gilded': 0,
      'clicked': False,
      'title': 'Вопрос: на каких платформах вы смотрите порно?',
      'link_flair_richtext': [],
      'subreddit_name_prefixed': 'r/rusAskReddit',
      'hidden': False,
      'pwls': 6,
      'link_flair_css_class': None,
      'downs': 0,
      'thumbnail_height': None,
      'top_awarded_type': None,
      'parent_whitelist_status': 'all_ads',
      'hide_score': False,
      'name': 't3_15g7ici',
      'quarantine': False,
      'link_flair_text_color': 'dark',
      'upvote_ratio': 0.69,
      'author_flair_background_color': None,
      'subreddit_type': 'public',
      'ups': 10,
      'total_awards_received': 0,
      'media_embed':

In [3]:
text_data = []

for item in data:
    if 'children' in item['data']:
        for child in item['data']['children']:
            if 'title' in child['data']:
                text_data.append(child['data']['title'])
            if 'body' in child['data']:
                text_data.append(child['data']['body'])
            if 'replies' in child['data'] and 'data' in child['data']['replies']:
                for reply_child in child['data']['replies']['data']['children']:
                    if 'body' in reply_child['data']:
                        text_data.append(reply_child['data']['body'])


In [4]:
text_data.append("сука, бля ты тупой")
text_data.append("хочу умереть , жизнь говно. Умрите все")

In [5]:
text_data

['Вопрос: на каких платформах вы смотрите порно?',
 '[deleted]',
 'Я год назад так здесь случайно зарегался 😅',
 'Я со словами " погоди, ща проверю" заходу в твой профиль, а там дет с молотом у носа. Я просто должен сказать, что этого я точно не ожидал',
 'охохххохо\nпроще сказать где я не смотрю..',
 'Ты вынудил мой фантазию включиться. Порно в игре UNO? penis reverse card.',
 'Xvideos , Pornhub , Порно365 , Порностудентки , Телеграм (Запрещенка ДВАЧ) и еще что-то .',
 'Икс видосики, иногда, прям выручают',
 'яндекс видео или тг :D',
 'Xvideo',
 'Pornhub, xvideos, xnxx, на твиттере подписан на нескольких нсфв артистов, ищу сливы с онлифанс на разных сайтах, pornomoloko, sexstudentki (мне батя этот сайт показал лол), в телеграмме подписан на кучу порно пабликов, xhamster, rule34, pornez, на newgrounds иногда, porntrex, thumbzilla, pornzog, erothots, thothub',
 'Батя показал😑😑🌚',
 'Я крайне рад что не нашел ни у кого furry34. Моя вера в человечество медленно возвращается',
 'Не пережива

In [6]:
banned_words = [
    "порно", "xvideos", "pornhub", "xnxx", "porn365", "порностудентки", "xhamster", "24video.com", "гейпоревоком", "se× studetki", "порно", "роzzиянам", "отлезали",
    "rule34", "pornez", "pornomoloko", "sexstudentki", "thumbzilla", "pornzog", "erothots", "сломали жизнь", "сука", "бля"
    "thothub", "furry34", "бойкиссер", "насилие", "смертники", "смерти",
    "пленными", "насилие", "оккупантов", "убивать", "жизнью", "хуйово", "кровь", "кровопицы",
    "кровавую", "ахуй", "аху[еий]", "porn",
    "cartoon porn", "teen",
    "shady sites",
    "illegal immigrant",
    "hack",
    "slippery slope",
    "censorship",
    "live-action porn",
    "traffics porn",
    "cult",
    "hack",
    "slippery slope",
    "censorship",
    "written consent",
    "live-action porn",
    "shotgun",
    "porn",
    "suck",
    "horny teenager",
    "earth",
    "sh#tty",
    "f#ckwads",
    "prohibition",
    "fuckgaloo",
    "alcapone",
    "religious senators",
    "enforce","shotgun",
    "porn",
    "american politicians",
    "suck-start a shotgun",
    "horny teenager",
    "earth",
    "sh#tty",
    "f#ckwads",
    "prohibition","CP",
    "electric fuckgaloo",
    "al capone", "fucked",
    "enforce", "dildo",
    "lmao","cum", "hentai", "rape", "riots", "cunts","pair of tits"
]

df = pd.DataFrame({"text": text_data})

In [7]:
df

,text
0,Вопрос: на каких платформах вы смотрите порно?
1,[deleted]
2,Я год назад так здесь случайно зарегался 😅
3,"Я со словами "" погоди, ща проверю"" заходу в тв..."
4,охохххохо\nпроще сказать где я не смотрю..
...,...
277,"I see, so they want a Prohibition 2: Electric ..."
278,Man these religious senators should not hold s...
279,How the fuck do you even enforce that lmao
280,"сука, бля ты тупой"


In [8]:
import re

def contains_banned_words(text):
    text_lower = text.lower()
    for word in banned_words:
        if re.search(word.lower(), text_lower):
            return True
    return False

In [9]:
df['label'] = df['text'].apply(contains_banned_words).astype(int)
df['text'] = df['text'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

tfidf_vectorizer = TfidfVectorizer()
X = tfidf_vectorizer.fit_transform(df['text'])
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
svm_model = SVC(kernel='linear', class_weight='balanced')
svm_model.fit(X_train, y_train)

SVC(class_weight='balanced', kernel='linear')

In [11]:
y_pred = svm_model.predict(X_test)

In [12]:
new_text_data = ["child porn"]

new_X = tfidf_vectorizer.transform(new_text_data)

predictions = svm_model.predict(new_X)
for text, label in zip(new_text_data, predictions):
    print(f"Text: {text}\nClassified as banned: {bool(label)}\n")

Text: child porn
Classified as banned: True



In [13]:
print(contains_banned_words("сексуальное насилие частое явление в доме"))

True


In [142]:
!pip install joblib 


[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
from joblib import dump

dump(svm_model, 'text_classifier.joblib')

['text_classifier.joblib']

In [18]:
dump(tfidf_vectorizer, 'tfidf_vectorizer.joblib')

['tfidf_vectorizer.joblib']

In [14]:
def retrain(new_text_data, new_ban_list = []):
    text_data.extend(new_text_data)
    banned_words.extend(new_ban_list)
    new_df = pd.DataFrame({"text": new_text_data})
    new_df['text'] = new_df['text'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

    new_df['label'] = new_df['text'].apply(contains_banned_words).astype(int)

    global df
    df = pd.concat([df, new_df], ignore_index=True)
    
    X = tfidf_vectorizer.fit_transform(df['text'])
    y = df['label']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    svm_model.fit(X_train, y_train)

    y_pred = svm_model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    for text, label in zip(new_text_data, y_pred):
        print(f"Text: {text}\nClassified as banned: {bool(label)}\n")
        
    # dump(svm_model, 'text_classifier.joblib')
    # dump(tfidf_vectorizer, 'tfidf_vectorizer.joblib')
    
    print(f"Retrained model accuracy: {accuracy}")

In [26]:
retrain(["Ебучее порно", "адское дрочево", "porn star"], ["Ебучее"])

Text: Ебучее порно
Classified as banned: True

Text: адское дрочево
Classified as banned: True

Text: porn star
Classified as banned: False

Retrained model accuracy: 0.890625
